# High Value Customer Identification

# 0.0 Imports

In [1]:
import os
import s3fs
import pickle
import sqlite3
import psycopg2 as pg
import regex    as re
import numpy    as np
import pandas   as pd

from sqlalchemy          import create_engine
from umap                import umap_         as umap
from sklearn             import ensemble      as en
from sklearn             import metrics       as m
from sklearn             import preprocessing as pp
from scipy.cluster       import hierarchy     as hc

## 0.2 Load Data

In [2]:
AWS_ACCESS_KEY_ID = os.environ.get( 'AWS_ACCESS_KEY_ID_CDS' )
AWS_SECRET_ACCESS_KEY = os.environ.get( 'AWS_SECRET_ACCESS_KEY_CDS' )

path_s3 = 's3://insiders-ds/Ecommerce.csv'

# Read files
df1 = pd.read_csv(path_s3, low_memory=False, encoding='cp1252')

# 1.0 Descrição dos dados

## 1.1 Rename Columns

In [4]:
# Drop error column
df1 = df1.drop(columns=['Unnamed: 8'], axis=1)

cols_news = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
              'unit_price', 'customer_id', 'country']

df1.columns = cols_news

## 1.4 Check NA

In [7]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Fillout NA

In [8]:
# Separate df into NA and not NA
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

# create df for reference
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())

# create sintetics customers id
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1)

# merge results of recerence into original df
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

# coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop extra columns
df1.drop(columns=['customer_id_x', 'customer_id_y'], axis=1, inplace=True)

In [9]:
# Description column is gonna be droped
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6 Change Types

In [10]:
# change invoice_date into date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# change customer_id into int 
df1['customer_id'] = df1['customer_id'].astype('int64')

In [11]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

# 2.0 Filtragem de variáveis

## 2.1 Seleção das colunas

In [20]:
df1.drop(columns='description', axis=1, inplace=True)

## 2.2 Filtragem das linhas

In [21]:
# remove unit_price bellow 0.030 (bellow this consider promo or return)
df1 = df1[df1['unit_price'] > 0.030]

# remove stock code with only categoricals variables (expect DCGSSBOY and DCGSSGIRL)
df1 = df1[~df1['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 
                                    'm', 'PADS', 'B', 'CRUK', 'BANK CHARGES'])]

# removes information from unidentified countries
df1 = df1[~df1['country'].isin(['European Community', 'Unspecified'])]

# bad users
df1 = df1[~df1['customer_id'].isin([16446, 12346])]
df1 = df1[~df1['invoice_no'].isin(['540815', '540818', 'C550456'])] # Customer 15749: duplicates orders

# 3.0 Feature Engineering

In [22]:
df3 = df1.copy()

## 3.1 Feature Engineering

In [23]:
# Time series variables
df3['day'] = df3['invoice_date'].dt.day

df3['month'] = df3['invoice_date'].dt.month

df3['year'] = df3['invoice_date'].dt.year

df3['week_of_year'] = df3['invoice_date'].dt.weekofyear

# separate dataframes for purchases and refunds
df3_purchases = df3[df3['quantity'] >= 0]
df3_returns = df3[df3['quantity'] < 0]

C:\Users\SamuelOliveiraAlvesd\AppData\Local\Temp\ipykernel_10876\1101900786.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df3['week_of_year'] = df3['invoice_date'].dt.weekofyear


In [24]:
# -------------------- Features based on RFM Model ---------------
# make data frame for reference
df_ref = df3[['customer_id']].drop_duplicates(ignore_index=True)


# Gross Revenue (Faturamento) | quantity * price
df3_purchases['gross_revenue'] = df3_purchases['quantity'] * df3_purchases['unit_price']


# Monetary - Total buys from each customer
df_monetary = df3_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

df_ref.isna().sum()

C:\Users\SamuelOliveiraAlvesd\AppData\Local\Temp\ipykernel_10876\1621994048.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_purchases['gross_revenue'] = df3_purchases['quantity'] * df3_purchases['unit_price']


customer_id       0
gross_revenue    67
dtype: int64

In [25]:
# Recency - Last day purchase for each customer
df_recency = df3_purchases[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df3['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency.drop(columns=['invoice_date'], inplace=True)
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')

df_ref.isna().sum()

customer_id       0
gross_revenue    67
recency_days     67
dtype: int64

In [26]:
# Quantity of purchased
df_qnt = (df3_purchases[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id')
                                             .count().reset_index()
                                             .rename(columns={'invoice_no': 'qty_invoices'}))

df_ref = pd.merge(df_ref, df_qnt, on='customer_id', how='left')

df_ref.isna().sum()

customer_id       0
gross_revenue    67
recency_days     67
qty_invoices     67
dtype: int64

In [27]:
# Quantity of items purchased
df_qnt_items = (df3_purchases[['customer_id', 'quantity']].groupby('customer_id')
                                             .sum().reset_index()
                                             .rename(columns={'quantity': 'qty_items'}))

df_ref = pd.merge(df_ref, df_qnt_items, on='customer_id', how='left')

df_ref.isna().sum()                                             

customer_id       0
gross_revenue    67
recency_days     67
qty_invoices     67
qty_items        67
dtype: int64

In [28]:
# Quantity of products purchased
df_qnt_products = (df3_purchases[['customer_id', 'stock_code']].groupby('customer_id')
                                             .count().reset_index()
                                             .rename(columns={'stock_code': 'qty_products'}))

df_ref = pd.merge(df_ref, df_qnt_products, on='customer_id', how='left')

df_ref.isna().sum()   

customer_id       0
gross_revenue    67
recency_days     67
qty_invoices     67
qty_items        67
qty_products     67
dtype: int64

In [29]:
# Avg ticket - average purchases per customer
df_avg_ticket = df3_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').mean().reset_index()
df_avg_ticket.columns = ['customer_id', 'avg_ticket']
df_ref = pd.merge(df_ref, df_avg_ticket, how='left', on='customer_id')

df_ref.isna().sum()

customer_id       0
gross_revenue    67
recency_days     67
qty_invoices     67
qty_items        67
qty_products     67
avg_ticket       67
dtype: int64

In [30]:
# Frequency - Frequency of purchases in aprox one year
df_aux = (df3_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates().groupby('customer_id')
                                                            .agg(buy_ = ('invoice_no', 'count') ).reset_index())

days = (df3_purchases['invoice_date'].max() - df3_purchases['invoice_date'].min()).days

df_aux['days_'] = days

# Calculate frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1)

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], how='left', on='customer_id')

df_ref.isna().sum()

customer_id       0
gross_revenue    67
recency_days     67
qty_invoices     67
qty_items        67
qty_products     67
avg_ticket       67
frequency        67
dtype: int64

In [31]:
# Number of returns
df_returns = df3_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qty_returns'})
df_returns['qty_returns'] = df_returns['qty_returns'] * -1

# Merge
df_ref = pd.merge(df_ref, df_returns, how='left', on='customer_id')
df_ref.loc[df_ref['qty_returns'].isna(), 'qty_returns'] = 0

df_ref.isna().sum()

customer_id       0
gross_revenue    67
recency_days     67
qty_invoices     67
qty_items        67
qty_products     67
avg_ticket       67
frequency        67
qty_returns       0
dtype: int64

In [32]:
# Ratio of number of returns
df_aux = df_ref[['customer_id', 'qty_items', 'qty_returns']].groupby('customer_id').sum().reset_index()
df_aux['returns_ratio'] = df_aux[['qty_items', 'qty_returns']].apply(lambda x: 0 if x['qty_returns'] <= 0 else 0 if x['qty_items'] <= 0 else x['qty_returns'] / x['qty_items'], axis=1)

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'returns_ratio']], how='left', on='customer_id')

df_ref = df_ref[df_ref['returns_ratio'] < 1.0] # cleaning customers with ratio above 1.0

df_ref.isna().sum()

customer_id       0
gross_revenue    67
recency_days     67
qty_invoices     67
qty_items        67
qty_products     67
avg_ticket       67
frequency        67
qty_returns       0
returns_ratio     0
dtype: int64

In [33]:
# Basket Size (Quantity)
df_aux = (df3_purchases[['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                         .agg(n_purchase=('invoice_no', 'nunique'), 
                                                         n_products=('quantity', 'sum')).reset_index())

# calculate basket size
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_basket_size']], how='left', on='customer_id')

df_ref.isna().sum()

customer_id         0
gross_revenue      67
recency_days       67
qty_invoices       67
qty_items          67
qty_products       67
avg_ticket         67
frequency          67
qty_returns         0
returns_ratio       0
avg_basket_size    67
dtype: int64

In [34]:
# Unique Basket Size
df_aux = (df3_purchases[['customer_id', 'invoice_no', 'stock_code']].groupby('customer_id')
                                                         .agg(n_purchase=('invoice_no', 'nunique'), 
                                                         n_products=('stock_code', 'nunique')).reset_index())

# calculate basket size
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], how='left', on='customer_id')

df_ref.isna().sum()

customer_id                0
gross_revenue             67
recency_days              67
qty_invoices              67
qty_items                 67
qty_products              67
avg_ticket                67
frequency                 67
qty_returns                0
returns_ratio              0
avg_basket_size           67
avg_unique_basket_size    67
dtype: int64

In [35]:
# Period of buy in each day

df_aux = df3_purchases[['customer_id', 'day', 'invoice_no']].drop_duplicates(subset='invoice_no')
df_aux['period_of_buy_day'] = df_aux.apply(lambda x: 'before_11' if x['day'] <= 10 else 'between_11_20' if x['day'] <= 20 else 'after_20', axis=1)
df_aux = df_aux[['customer_id', 'period_of_buy_day', 'day']].groupby(['customer_id', 'period_of_buy_day']).count().sort_values('day', ascending=False).reset_index().drop_duplicates(subset='customer_id')

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'period_of_buy_day']], how='left', on='customer_id')

df_ref.isna().sum()

customer_id                0
gross_revenue             67
recency_days              67
qty_invoices              67
qty_items                 67
qty_products              67
avg_ticket                67
frequency                 67
qty_returns                0
returns_ratio              0
avg_basket_size           67
avg_unique_basket_size    67
period_of_buy_day         67
dtype: int64

In [36]:
# Period of buy in each quarter

df_aux = df3_purchases[['customer_id', 'month', 'invoice_no']].drop_duplicates(subset='invoice_no')
df_aux['period_of_buy_quarter'] = df_aux.apply(lambda x: 1 if x['month'] <= 3 else 2 if x['month'] <= 6 else 3 if x['month'] <= 9 else 4, axis=1)
df_aux = df_aux[['customer_id', 'period_of_buy_quarter', 'month']].groupby(['customer_id', 'period_of_buy_quarter']).count().sort_values('month', ascending=False).reset_index().drop_duplicates(subset='customer_id')

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'period_of_buy_quarter']], how='left', on='customer_id')

df_ref.isna().sum()

customer_id                0
gross_revenue             67
recency_days              67
qty_invoices              67
qty_items                 67
qty_products              67
avg_ticket                67
frequency                 67
qty_returns                0
returns_ratio              0
avg_basket_size           67
avg_unique_basket_size    67
period_of_buy_day         67
period_of_buy_quarter     67
dtype: int64

In [37]:
# Country
df_aux = df3[['customer_id', 'country']].drop_duplicates(subset='customer_id')

# Merge
df_ref = pd.merge(df_ref, df_aux, how='left', on='customer_id')

df_ref.isna().sum()

customer_id                0
gross_revenue             67
recency_days              67
qty_invoices              67
qty_items                 67
qty_products              67
avg_ticket                67
frequency                 67
qty_returns                0
returns_ratio              0
avg_basket_size           67
avg_unique_basket_size    67
period_of_buy_day         67
period_of_buy_quarter     67
country                    0
dtype: int64

In [38]:
df_ref = df_ref.dropna()

df_ref['period_of_buy_quarter'] = df_ref['period_of_buy_quarter'].astype('int64')

df_ref.isna().sum()

customer_id               0
gross_revenue             0
recency_days              0
qty_invoices              0
qty_items                 0
qty_products              0
avg_ticket                0
frequency                 0
qty_returns               0
returns_ratio             0
avg_basket_size           0
avg_unique_basket_size    0
period_of_buy_day         0
period_of_buy_quarter     0
country                   0
dtype: int64

# 5.0 Preparação dos dados

In [44]:
df5 = df_ref.drop(columns=['customer_id'], axis=1).copy()

In [45]:
# Make rescaling for data organization analysis
mms = pp.MinMaxScaler()
fs = s3fs.S3FileSystem( anon=False, key=AWS_ACCESS_KEY_ID , secret=AWS_SECRET_ACCESS_KEY )


gross_revenue_scaler = pickle.load(fs.open( 's3://insiders-ds/gross_revenue_scaler.pkl', 'rb' ))
df5['gross_revenue'] = gross_revenue_scaler.transform(df5[['gross_revenue']].values)


recency_days_scaler = pickle.load(fs.open( 's3://insiders-ds/recency_days_scaler.pkl', 'rb' ))
df5['recency_days'] = recency_days_scaler.transform(df5[['recency_days']].values)


qty_invoices_scaler = pickle.load(fs.open( 's3://insiders-ds/qty_invoices_scaler.pkl', 'rb' ))
df5['qty_invoices'] = qty_invoices_scaler.transform(df5[['qty_invoices']].values)


qty_items_scaler = pickle.load(fs.open( 's3://insiders-ds/qty_items_scaler.pkl', 'rb' ))
df5['qty_items'] = qty_items_scaler.transform(df5[['qty_items']].values)


qty_products_scaler = pickle.load(fs.open( 's3://insiders-ds/qty_products_scaler.pkl', 'rb' ))
df5['qty_products'] = qty_products_scaler.transform(df5[['qty_products']].values)


avg_ticket_scaler = pickle.load(fs.open( 's3://insiders-ds/avg_ticket_scaler.pkl', 'rb' ))
df5['avg_ticket'] = avg_ticket_scaler.transform(df5[['avg_ticket']].values)


frequency_scaler = pickle.load(fs.open( 's3://insiders-ds/frequency_scaler.pkl', 'rb' ))
df5['frequency'] = frequency_scaler.transform(df5[['frequency']].values)


qty_returns_scaler = pickle.load(fs.open( 's3://insiders-ds/qty_returns_scaler.pkl', 'rb' ))
df5['qty_returns'] = qty_returns_scaler.transform(df5[['qty_returns']].values)


returns_ratio_scaler = pickle.load(fs.open( 's3://insiders-ds/returns_ratio_scaler.pkl', 'rb' ))
df5['returns_ratio'] = returns_ratio_scaler.transform(df5[['returns_ratio']].values)


avg_basket_size_scaler = pickle.load(fs.open( 's3://insiders-ds/avg_basket_size_scaler.pkl', 'rb' ))
df5['avg_basket_size'] = avg_basket_size_scaler.transform(df5[['avg_basket_size']].values)


avg_unique_basket_size_scaler = pickle.load(fs.open( 's3://insiders-ds/avg_unique_basket_size_scaler.pkl', 'rb' ))
df5['avg_unique_basket_size'] = avg_unique_basket_size_scaler.transform(df5[['avg_unique_basket_size']].values)


period_of_buy_quarter_scaler = pickle.load(fs.open( 's3://insiders-ds/period_of_buy_quarter_scaler.pkl', 'rb' ))
df5['period_of_buy_quarter'] = period_of_buy_quarter_scaler.transform(df5[['period_of_buy_quarter']].values)

# Frequency Encoder
fe_period_of_buy_day = pickle.load(fs.open( 's3://insiders-ds/fe_period_of_buy_day_enc.pkl', 'rb' ))
df5.loc[:, 'period_of_buy_day'] = df5['period_of_buy_day'].map(fe_period_of_buy_day)

fe_country = pickle.load(fs.open( 's3://insiders-ds/fe_country_enc.pkl', 'rb' ))
df5.loc[:, 'country'] = df5['country'].map(fe_country)

# 6.0 Estudo do espaço

In [47]:
X = df5.copy()

## 6.4 Tree-based Embedding

### 6.4.2 Tree Embedding

In [48]:
# training dataset
X = df5.drop(columns=['gross_revenue'], axis=1).copy()
y = df_ref['gross_revenue']

rf_model = pickle.load(fs.open( 's3://insiders-ds/rf_model.pkl', 'rb' ))

# make dataframe for leaf
df_leaf = pd.DataFrame(rf_model.apply(X))

# reducer dimensionality
reducer = pickle.load(fs.open( 's3://insiders-ds/umap_reducer.pkl', 'rb' ))

embedding = reducer.transform(df_leaf)

# Embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:,0]
df_tree['embedding_y'] = embedding[:,1]

# 9.0 Machine Learning Modeling

## 9.1 HC

In [ ]:
X = df_tree.copy()

In [ ]:
# model definition and training
hc_model = hc.linkage(X, 'ward')

# Model predict
labels = hc.fcluster(hc_model, 8, criterion='maxclust')

# Metrics
sil = m.silhouette_score(X, labels)

# 10.0 Cluster Analysis

In [ ]:
df10 = X.copy()
df10['cluster'] = labels
df10.head()

,embedding_x,embedding_y,cluster
0,-7.710938,7.817896,1
1,-7.098626,9.921972,1
2,0.227572,17.649805,5
3,2.463838,-6.946425,3
4,20.882982,7.314658,7


## 10.4 Cluster Profile

In [ ]:
df10 = df_ref.reset_index( drop=True).copy()
df10['cluster'] = labels

# Number of customers
df_cluster = df10[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = 100 * (df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# Avg Gross revenue
df_avg_gross_revenue = df10[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# Avg recency days
df_avg_recency_days = df10[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

# Avg qty products
df_avg_recency_days = df10[['qty_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

# Frequency
df_invoice_no = df10[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_invoice_no, how='inner', on='cluster')

# Returns ratio
df_ticket = df10[['returns_ratio', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_ticket, how='inner', on='cluster')

# Cluster names from RFM Model
names = {1 : "Insiders",
         5 : "Loyal Customers",
         2 : "Potential Loyalists",
         4 : "Promissing ",
         3 : "Need Attention",
         8 : "Hibernating",
         7 : "Price Sensitive",
         6 : "Churn"}
df_cluster['cluster_name'] = df_cluster['cluster'].map(names)

df_cluster[['cluster', 'cluster_name', 'customer_id', 'perc_customer', 'gross_revenue', 'recency_days', 'qty_products',	'frequency', 'returns_ratio']].sort_values('gross_revenue', ascending=False)

,cluster,cluster_name,customer_id,perc_customer,gross_revenue,recency_days,qty_products,frequency,returns_ratio
0,1,Insiders,416,7.323944,8299.488822,38.004808,244.173077,0.027744,0.014972
4,5,Loyal Customers,480,8.450704,5976.284792,40.052083,276.172917,0.030870,0.014057
1,2,Potential Loyalists,1045,18.397887,1864.093828,87.016268,144.840191,0.009118,0.011055
3,4,Promissing,413,7.271127,1006.376731,106.910412,82.353511,0.006745,0.010866
2,3,Need Attention,647,11.390845,722.784590,97.290572,55.729521,0.006588,0.008663
7,8,Hibernating,1400,24.647887,467.971243,135.263571,38.179286,0.004370,0.011303
6,7,Price Sensitive,660,11.619718,232.572364,181.074242,21.440909,0.003246,0.008394
5,6,Churn,619,10.897887,57.489305,196.901454,6.521809,0.002789,0.002412


# 12.0 Deploy model to production

In [ ]:
df10.dtypes

customer_id                 int64
gross_revenue             float64
recency_days              float64
qty_invoices              float64
qty_items                 float64
qty_products              float64
avg_ticket                float64
frequency                 float64
qty_returns               float64
returns_ratio             float64
avg_basket_size           float64
avg_unique_basket_size    float64
period_of_buy_day          object
period_of_buy_quarter       int64
country                    object
cluster                     int32
dtype: object

In [ ]:
df10['recency_days'] = df10['recency_days'].astype('int64')
df10['qty_invoices'] = df10['qty_invoices'].astype('int64')
df10['qty_items'] = df10['qty_items'].astype('int64')
df10['qty_products'] = df10['qty_products'].astype('int64')
df10['qty_returns'] = df10['qty_returns'].astype('int64')
df10['cluster'] = df10['cluster'].astype('int64')

In [ ]:
# Credentials
db_acess =  open('/.aws/dbacess', 'r')
host = db_acess.readlines()[0][:-1]

db_acess =  open('/.aws/dbacess', 'r')
port = db_acess.readlines()[1][:-1]

db_acess =  open('/.aws/dbacess', 'r')
database = db_acess.readlines()[2][:-1]

db_acess =  open('/.aws/dbacess', 'r')
username = db_acess.readlines()[3][:-1]

db_acess =  open('/.aws/dbacess', 'r')
password = db_acess.readlines()[4]


# Acess database
conn = pg.connect(user=username,
                  password=password,
                  host=host,
                  port=port,
                  database=database)

# Make a cursor into database
cursor = conn.cursor()

# Verify all schemas
query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
    """

cursor.execute(query_schema)
record = cursor.fetchall()
record

[('pg_toast',), ('pg_catalog',), ('information_schema',), ('public',)]

In [ ]:
# Verify tables from the schema
query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname='public'
    """

cursor.execute(query_tables)
record = cursor.fetchall()
record

[('insiders',)]

In [ ]:
# Collect tables into datafram
query_tables_insurance = """
    SELECT *
    FROM insiders
    """

df_test = pd.read_sql(query_tables_insurance, con=conn)

# close connection into database
cursor.close()
conn.close()

df_test.head()

f:\SamuelOliveiraAlvesd\Anaconda3\envs\clustering_insiders\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,gross_revenue,recency_days,qty_invoices,qty_items,qty_products,avg_ticket,frequency,qty_returns,returns_ratio,avg_basket_size,avg_unique_basket_size,period_of_buy_day,period_of_buy_quarter,country,cluster
0,17850,5391.21,372,34,1733,297,18.152222,0.091153,40,0.023081,50.970590,0.617647,after_20,4,United Kingdom,1
1,13047,3232.59,56,9,1390,171,18.904036,0.024129,35,0.025180,154.444440,11.666667,after_20,4,United Kingdom,1
2,12583,6705.38,2,15,5028,232,28.902500,0.040214,50,0.009944,335.200000,7.600000,before_11,4,France,5
3,13748,948.25,95,5,439,28,33.866070,0.013405,0,0.000000,87.800000,4.800000,before_11,3,United Kingdom,3
4,15100,876.00,333,3,80,3,292.000000,0.008043,22,0.275000,26.666666,0.333333,before_11,4,United Kingdom,7


In [ ]:
#saving into db
#df10.to_sql( 'insiders', con=conn, if_exists='append', index=False )